In [1]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 

D:\Anaconda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)

Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean


In [3]:
import math 
def cos(x):
    return(-10+(5-x)**2)/20

In [4]:
def data_create(seed,time_stamp):
  np.random.seed(seed)
  x = np.array(range(0, time_stamp))
  y = np.array([cos(i) for i in x]) #generate coefficient

  base_DAG=np.zeros((5, 5))
  base_DAG[0,4]=y[0]##edited to be coeffcient with error
  base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
  X_all = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
  for i in range(1,time_stamp):
      base_DAG[0,4]=y[i]
      base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
      X = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
      X_all=np.append(X_all,X,axis=0)
  return X_all

In [5]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
n_times=30
seed_list=np.random.randint(1, 1000000, size=n_times)
seed=seed_list[0]
X_all=data_create(seed,time_stamp) #create data

## piecewise ANOCA

In [6]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import pickle
import os
import random

import torch.optim as optim
from torch.optim import lr_scheduler
import math
from utils import *

from multiprocessing import Pool
import multiprocessing
n_cores = multiprocessing.cpu_count()
from numpy.random import randn
from random import seed as rseed
from numpy.random import seed as npseed


In [7]:
# compute constraint h(A) value
def _h_A(A, m):
    expm_A = matrix_poly(A*A, m)
    h_A = torch.trace(expm_A) - m
    return h_A

prox_plus = torch.nn.Threshold(0.,0.)

def stau(w, tau):
    w1 = prox_plus(torch.abs(w)-tau)
    return torch.sign(w)*w1


def update_optimizer(optimizer, original_lr, c_A):
    '''related LR to c_A, whenever c_A gets big, reduce LR proportionally'''
    MAX_LR = 1e-2
    MIN_LR = 1e-4

    estimated_lr = original_lr / (math.log10(c_A) + 1e-10)
    if estimated_lr > MAX_LR:
        lr = MAX_LR
    elif estimated_lr < MIN_LR:
        lr = MIN_LR
    else:
        lr = estimated_lr

    # set LR
    for parame_group in optimizer.param_groups:
        parame_group['lr'] = lr

    return optimizer, lr

#===================================
# training:
#===================================

def train(epoch, lambda_A, c_A, optimizer,old_lr):
    t = time.time()
    nll_train = []
    kl_train = []
    mse_train = []
    shd_trian = []

    encoder.train()
    decoder.train()
    scheduler.step()


    # update optimizer
    optimizer, lr = update_optimizer(optimizer, old_lr, c_A)


    for batch_idx, (data, relations) in enumerate(train_loader):

#         if args.cuda:
#             data, relations = data.cuda(), relations.cuda()
        data, relations = Variable(data).double(), Variable(relations).double()

        # reshape data
        relations = relations.unsqueeze(2)

        optimizer.zero_grad()

        enc_x, logits, origin_A, adj_A_tilt_encoder, z_gap, z_positive, myA, Wa = encoder(data, rel_rec, rel_send)  # logits is of size: [num_sims, z_dims]
        edges = logits

        dec_x, output, adj_A_tilt_decoder = decoder(data, edges,d * x_dims, rel_rec, rel_send, origin_A, adj_A_tilt_encoder, Wa)

        if torch.sum(output != output):
            print('nan error\n')

        target = data
        preds = output
        variance = 0.

        # reconstruction accuracy loss
        loss_nll = nll_gaussian(preds, target, variance)

        # KL loss
        loss_kl = kl_gaussian(logits)

        # ELBO loss:
        loss = loss_kl + loss_nll

        # add A loss
        one_adj_A = origin_A # torch.mean(adj_A_tilt_decoder, dim =0)
        sparse_loss = tau_A * torch.sum(torch.abs(one_adj_A))

        # compute h(A)
        h_A = _h_A(origin_A, d)
        loss += lambda_A * h_A + 0.5 * c_A * h_A * h_A + 100. * torch.trace(origin_A*origin_A) + sparse_loss #+  0.01 * torch.sum(variance * variance)


        loss.backward()
        loss = optimizer.step()

        myA.data = stau(myA.data, tau_A*lr)


        mse_train.append(F.mse_loss(preds, target).item())
        nll_train.append(loss_nll.item())
        kl_train.append(loss_kl.item())

    #print(h_A.item())

    return np.mean(np.mean(kl_train)  + np.mean(nll_train)), np.mean(nll_train), np.mean(mse_train), origin_A, optimizer, lr



In [8]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
x_dims = 1 # The number of input dimensions: default 1.
z_dims = d # The number of latent variable dimensions: default the same as variable size.
epochs = 200 # Number of epochs to train.
batch_size = 10 # Number of samples per batch. note: should be divisible by sample size, otherwise throw an error.
n_var=5

n_times=30 #no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list=np.zeros((n_times,time_stamp,n_var,n_var))
B_list=np.zeros((n_times,d, d))
FDR_total=[]
TPR_total=[]
SHD_total=[]
time_list=[]
for replicate in range(n_times):
  seed=seed_list[replicate]
  print(seed)
  X_all=data_create(seed,time_stamp) #create data
  average_list=np.zeros((time_stamp,d, d))
  FDR_list_piece=[]
  TPR_list_piece=[]
  SHD_list_piece=[]
  base_DAG=np.zeros((5, 5))
  ####estimate at each time_stamp####
  timestart=time.time()
  for j in range(time_stamp):
  # ----------- Configurations:
      k_max_iter = int(1e2) # The max iteration number for searching parameters.
      original_lr = 3e-3  # Initial learning rate.
      encoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      decoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      temp = 0.5 # Temperature for Gumbel softmax.
      factor = True # Factor graph model.
      encoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      decoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      tau_A = 0. # Coefficient for L-1 norm of matrix B.
      lambda1 = 0. # Coefficient for DAG constraint h1(B).
      c_B = 1 # Coefficient for absolute value h1(B).
      h1_tol = 1e-8 # The tolerance of error of h1(B) to zero.
      lr_decay = 200 # After how many epochs to decay LR by a factor of gamma. 
      gamma = 1.0 # LR decay factor. 
      ######################


      X=X_all[(j*30):(j*30+30),:]


      np.random.seed(seed)
      random.seed(seed)
      torch.manual_seed(seed)
      feat_train = torch.FloatTensor(X)
      feat_valid = torch.FloatTensor(X)
      feat_test = torch.FloatTensor(X)

      # Reconstruct itself
      train_data = TensorDataset(feat_train, feat_train)
      valid_data = TensorDataset(feat_valid, feat_train)
      test_data = TensorDataset(feat_test, feat_train)

      train_loader = DataLoader(train_data, batch_size = batch_size)
      valid_loader = DataLoader(valid_data, batch_size = batch_size)
      test_loader = DataLoader(test_data, batch_size = batch_size)

      # ----------- Load modules:
      off_diag = np.ones([d, d]) - np.eye(d) # Generate off-diagonal interaction graph
      rel_rec = np.array(encode_onehot(np.where(off_diag)[1]), dtype = np.float64)
      rel_send = np.array(encode_onehot(np.where(off_diag)[0]), dtype = np.float64)
      rel_rec = torch.DoubleTensor(rel_rec)
      rel_send = torch.DoubleTensor(rel_send)
      adj_A = np.zeros((d, d)) # Add adjacency matrix

      encoder = MLPEncoder(d * x_dims, x_dims, encoder_hidden,
                              int(z_dims), adj_A,
                              batch_size = batch_size,
                              do_prob = encoder_dropout, factor = factor).double()
      decoder = MLPDecoder(d * x_dims,
                              z_dims, x_dims, encoder,
                              data_variable_size = d,
                              batch_size = batch_size,
                              n_hid=decoder_hidden,
                              do_prob=decoder_dropout).double()

      # ----------- Set up optimizer:
      optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr = original_lr)
      scheduler = lr_scheduler.StepLR(optimizer, step_size = lr_decay,
                                      gamma = gamma)

      rel_rec = Variable(rel_rec)
      rel_send = Variable(rel_send)

      # ----------- Main:
      best_ELBO_loss = np.inf
      best_NLL_loss = np.inf
      best_MSE_loss = np.inf
      h1_B_new = torch.tensor(1.)
      h2_B_new = 1
      h1_B_old = np.inf
      h2_B_old = np.inf
      lr = original_lr
    

      try:
          for step_k in range(k_max_iter):
              while c_B< 1e+20:
                  for epoch in range(epochs):
                      old_lr = lr 
                      ELBO_loss, NLL_loss, MSE_loss, origin_B, optimizer, lr = train(epoch, lambda1, c_B, optimizer, old_lr)

                      if ELBO_loss < best_ELBO_loss:
                          best_ELBO_loss = ELBO_loss

                      if NLL_loss < best_NLL_loss:
                          best_NLL_loss = NLL_loss

                      if MSE_loss < best_MSE_loss:
                          best_MSE_loss = MSE_loss

                  if ELBO_loss > 2 * best_ELBO_loss:
                      break

                  # Update parameters
                  B_new = origin_B.data.clone()
                  h1_B_new =  _h_A(B_new,d)
                  if h1_B_new.item() > 0.25 * h1_B_old:
                      c_B *= 10
                  else:
                      break

              # Update parameters    
              h1_B_old = h1_B_new.item()
              lambda1 += c_B * h1_B_new.item()

              if h1_B_new.item() <= h1_tol:
                  break

      except KeyboardInterrupt:
          print('KeyboardInterrupt')

      predB = np.matrix(origin_B.data.clone().numpy())
      print('Best ELBO Loss :', best_ELBO_loss)
      print('Best NLL Loss :', best_NLL_loss)
      print('Best MSE Loss :', best_MSE_loss)
      #calculate_effect(predB)
      print(j)
      average_list[j,:,:]=predB



  average_coef_list[replicate,:,:,:]=average_list #average coef save to matrix
  np.save("quadra_10_30_DAGGNN",average_coef_list)
#   timeend=time.time()
#   time_list.append(timeend-timestart)
  #print(timeend-timestart)
  #####write at every epoch
  #df.to_csv("cos_rep10.csv")
  print(replicate)

913812


D:\Anaconda\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Best ELBO Loss : 0.0008028818758231104
Best NLL Loss : 1.396529060025426e-06
Best MSE Loss : 5.586116240101703e-07
0
Best ELBO Loss : 0.0004869945321313237
Best NLL Loss : 1.6117127664891235e-06
Best MSE Loss : 6.446851065956493e-07
1
Best ELBO Loss : 0.0009032578405923822
Best NLL Loss : 9.232724249376635e-07
Best MSE Loss : 3.6930896997506535e-07
2
Best ELBO Loss : 0.0003319975912376866
Best NLL Loss : 1.381117001572045e-06
Best MSE Loss : 5.52446800628818e-07
3
Best ELBO Loss : 0.000427929184836936
Best NLL Loss : 2.066099194619964e-07
Best MSE Loss : 8.264396778479857e-08
4
Best ELBO Loss : 0.0005165782556750545
Best NLL Loss : 5.720364132077351e-07
Best MSE Loss : 2.28814565283094e-07
5
Best ELBO Loss : 0.000513425403666668
Best NLL Loss : 3.04467629599172e-07
Best MSE Loss : 1.217870518396688e-07
6
Best ELBO Loss : 0.0005061683321183793
Best NLL Loss : 3.7649256739375397e-06
Best MSE Loss : 1.505970269575016e-06
7
Best ELBO Loss : 0.0008816298461211175
Best NLL Loss : 7.664065469

Best ELBO Loss : 0.0012330049890966345
Best NLL Loss : 1.3005260809715826e-06
Best MSE Loss : 5.202104323886331e-07
0
Best ELBO Loss : 0.0011211911997177975
Best NLL Loss : 2.7349457418526544e-06
Best MSE Loss : 1.0939782967410618e-06
1
Best ELBO Loss : 0.0012511417228755054
Best NLL Loss : 4.698063561673714e-06
Best MSE Loss : 1.879225424669486e-06
2
Best ELBO Loss : 0.0007054529552818754
Best NLL Loss : 1.6800400029559562e-06
Best MSE Loss : 6.720160011823824e-07
3
Best ELBO Loss : 0.0006543596111601463
Best NLL Loss : 1.1809714225710039e-06
Best MSE Loss : 4.7238856902840155e-07
4
Best ELBO Loss : 0.0007034594596036062
Best NLL Loss : 1.5107722332025414e-06
Best MSE Loss : 6.043088932810166e-07
5
Best ELBO Loss : 0.0006331398699433002
Best NLL Loss : 5.725708048217591e-07
Best MSE Loss : 2.2902832192870363e-07
6
Best ELBO Loss : 0.0005126540378834124
Best NLL Loss : 8.661882541603001e-07
Best MSE Loss : 3.4647530166412e-07
7
Best ELBO Loss : 0.0011106148394449185
Best NLL Loss : 2.3

Best ELBO Loss : 0.0012221821885688216
Best NLL Loss : 2.219089691315153e-06
Best MSE Loss : 8.87635876526061e-07
0
Best ELBO Loss : 0.0006586056435882883
Best NLL Loss : 1.5057214031550937e-06
Best MSE Loss : 6.022885612620374e-07
1
Best ELBO Loss : 0.001032781399533859
Best NLL Loss : 1.864997255105786e-06
Best MSE Loss : 7.459989020423145e-07
2
Best ELBO Loss : 0.0006164101662709918
Best NLL Loss : 1.2734208743546845e-06
Best MSE Loss : 5.093683497418738e-07
3
Best ELBO Loss : 0.0006533815850950291
Best NLL Loss : 1.487040090305251e-06
Best MSE Loss : 5.948160361221004e-07
4
Best ELBO Loss : 0.000712528413241238
Best NLL Loss : 9.94192761125477e-07
Best MSE Loss : 3.9767710445019073e-07
5
Best ELBO Loss : 0.0005761333932608864
Best NLL Loss : 7.073765497717665e-07
Best MSE Loss : 2.829506199087066e-07
6
Best ELBO Loss : 0.0006614986285791356
Best NLL Loss : 6.770718365783903e-07
Best MSE Loss : 2.708287346313561e-07
7
Best ELBO Loss : 0.0013359520236428716
Best NLL Loss : 2.87000076

Best ELBO Loss : 0.0008643114219520683
Best NLL Loss : 1.4742528472889771e-06
Best MSE Loss : 5.897011389155908e-07
0
Best ELBO Loss : 0.0008131297670850709
Best NLL Loss : 2.0093993394552614e-05
Best MSE Loss : 8.037597357821046e-06
1
Best ELBO Loss : 0.0007732575815526179
Best NLL Loss : 1.4348694829511336e-06
Best MSE Loss : 5.739477931804534e-07
2
Best ELBO Loss : 0.0005625755806232161
Best NLL Loss : 3.3849051796880155e-06
Best MSE Loss : 1.3539620718752064e-06
3
Best ELBO Loss : 0.0005380732608480129
Best NLL Loss : 8.577977121377118e-07
Best MSE Loss : 3.4311908485508477e-07
4
Best ELBO Loss : 0.0005720196900097363
Best NLL Loss : 4.681172693354309e-07
Best MSE Loss : 1.8724690773417236e-07
5
Best ELBO Loss : 0.0004992104342059235
Best NLL Loss : 3.8148919908080564e-07
Best MSE Loss : 1.5259567963232224e-07
6
Best ELBO Loss : 0.0005597052790895112
Best NLL Loss : 3.3693440277345125e-06
Best MSE Loss : 1.3477376110938049e-06
7
Best ELBO Loss : 0.0012571175642743968
Best NLL Loss 

Best ELBO Loss : 0.002772490490095354
Best NLL Loss : 7.099315510780835e-06
Best MSE Loss : 2.8397262043123335e-06
0
Best ELBO Loss : 0.0027935475325636143
Best NLL Loss : 8.400548972089854e-06
Best MSE Loss : 3.360219588835942e-06
1
Best ELBO Loss : 0.0022298386869171833
Best NLL Loss : 6.601473603510162e-06
Best MSE Loss : 2.6405894414040642e-06
2
Best ELBO Loss : 0.002429741701362328
Best NLL Loss : 6.6401340187584965e-06
Best MSE Loss : 2.6560536075033983e-06
3
Best ELBO Loss : 0.0009917828749672845
Best NLL Loss : 1.646992231214934e-06
Best MSE Loss : 6.587968924859735e-07
4
Best ELBO Loss : 0.0021494340884671993
Best NLL Loss : 5.221158894416057e-06
Best MSE Loss : 2.088463557766423e-06
5
Best ELBO Loss : 0.0026685757415123553
Best NLL Loss : 7.808971091172185e-06
Best MSE Loss : 3.1235884364688747e-06
6
Best ELBO Loss : 0.002024478014975604
Best NLL Loss : 5.35759177992102e-06
Best MSE Loss : 2.1430367119684085e-06
7
Best ELBO Loss : 0.0019228124374937037
Best NLL Loss : 5.95499